In [9]:
import pandas as pd
import lib.tritime as tt

badges = tt.get_badges()

In [15]:
bcol, dcol, tin, tout, tdiff = [], [], [], [], []
for badge_num, badge in badges.items():
    dn = badge['display_name']
    punch_data = tt.read_punches(badge_num)
    for p in punch_data:
        if 'ts_in' not in p or 'ts_out' not in p:
            continue  # And flag as an error?
        bcol.append(badge_num)
        dcol.append(dn)
        tin.append(p['ts_in'])
        tout.append(p['ts_out'])
        tdiff.append(p['duration'])

tdf = pd.DataFrame({'badge': bcol, 'display_name': dcol,
                   'time_in': tin, 'time_out': tout, 'duration': tdiff})

tdf['time_in'] = pd.to_datetime(tdf['time_in'])
tdf['week_number'] = tdf['time_in'].dt.isocalendar().week
tdf.to_csv('test.csv', index=False)

AttributeError: Can only use .dt accessor with datetimelike values